#### This example shows how to use hydrolinking methods to address a point to the NHD High Resolution.

#### For this example we use a point near the Red Cedar River in East Lansing, MI.

##### First set required parameters.
Note: The input parameters are the same for NHDPlusV2.1 and NHD High Resolution hydrolinking methods.

Required parameters include:

    *input_identifier: str, user supplied identifier
    *input_lat: float, latitude of the point to be hydrolinked
    *input_lon: float, longitude of the point to be hydrolinked
    
Other optional parameters include:

    *input_crs: int, coordinate reference system, default is 4269 (NAD83)
    *water_name: str, name of the water feature that the point is intended to be linked to
    *buffer_m = int, search distance in meters, default is 1000, max is 2000
    *method = str, link data to 'closest' or 'name_match' (closest having a matching water name), default 'name_match'
    *hydro_type = str, link to 'waterbody' or 'flowline' NHD features, default 'flowline'

In [1]:
#identifier -> number or string
input_identifier = 1
#latitude
input_lat = 42.7284
#longitude
input_lon = -84.5026
#coordinate system
input_crs = 4269  
#In this case we want the point to link to the Red Cedar River so water_name is a stream name
water_name = 'The Red Cedar River'
#buffer
buffer_m = 2000


method = 'name_match'
hydro_type = 'flowline'

First the module for National Hydrography Dataset High Resolution is imported.  This HydroLink module is called nhd_hr.

In [2]:
#Import module from local folder hydrolink
from hydrolink import nhd_hr
import warnings; warnings.simplefilter('ignore')

In [3]:
# Initiate a Python object, this tries reprojecting data to nad83 and checks if data within bounding box of U.S.
hydrolink = nhd_hr.HighResPoint(input_identifier, input_lat, input_lon, water_name=water_name, input_crs=input_crs, buffer_m=buffer_m)

# Runs the HydroLink process using name_match method and flowline hydro_type.  
#This writes data out to a CSV in working directory named 'nhdhr_hydrolink_output.csv'
hydrolink.hydrolink_method(method=method, hydro_type=hydro_type)

In [4]:
#JSON representing all flowlines within buffer of the input coordinates (point to HydroLink)
hydrolink.flowlines_json

{'displayFieldName': 'gnis_name',
 'hasM': True,
 'fieldAliases': {'gnis_name': 'GNIS_NAME',
  'lengthkm': 'LENGTHKM',
  'permanent_identifier': 'PERMANENT_IDENTIFIER',
  'reachcode': 'REACHCODE'},
 'geometryType': 'esriGeometryPolyline',
 'spatialReference': {'wkid': 4269, 'latestWkid': 4269},
 'fields': [{'name': 'gnis_name',
   'type': 'esriFieldTypeString',
   'alias': 'GNIS_NAME',
   'length': 65},
  {'name': 'lengthkm', 'type': 'esriFieldTypeDouble', 'alias': 'LENGTHKM'},
  {'name': 'permanent_identifier',
   'type': 'esriFieldTypeString',
   'alias': 'PERMANENT_IDENTIFIER',
   'length': 40},
  {'name': 'reachcode',
   'type': 'esriFieldTypeString',
   'alias': 'REACHCODE',
   'length': 14}],
 'features': [{'attributes': {'gnis_name': None,
    'lengthkm': 0.011,
    'permanent_identifier': '152093560',
    'reachcode': '04050004002359'},
   'geometry': {'hasM': True,
    'paths': [[[-84.50938251259817, 42.72651486701329, 1.83965],
      [-84.50947937926469, 42.726588067013154, 0

In [5]:
#HydroLink data for all flowlines within buffer, these data are evaluated to find closest flowline 
#with a matching name (using a fuzzy match approach)
hydrolink.flowlines_data

[{'gnis_name': None,
  'lengthkm': 0.011,
  'permanent_identifier': '152093560',
  'reachcode': '04050004002359',
  'flowline name similarity message': 'no GNIS name',
  'cleaned source water name': 'the red cedar river',
  'flowline name similarity': 0,
  'meters from flowline': 590.9925501612136,
  'nhdhr flowline measure': 1.83965},
 {'gnis_name': 'Red Cedar River',
  'lengthkm': 1.746,
  'permanent_identifier': '152093412',
  'reachcode': '04050004000126',
  'cleaned source water name': 'the red cedar river',
  'flowline name similarity': 0.8823529411764706,
  'flowline name similarity message': 'most likely match, based on fuzzy match',
  'meters from flowline': 595.535732278204,
  'nhdhr flowline measure': 19.83811},
 {'gnis_name': None,
  'lengthkm': 0.455,
  'permanent_identifier': '152091798',
  'reachcode': '04050004002359',
  'flowline name similarity message': 'no GNIS name',
  'cleaned source water name': 'the red cedar river',
  'flowline name similarity': 0,
  'meters fr

In [6]:
#HydroLink Data for the selected flowline.
#The name_match approach uses the closest flowline with a name match (meeting a specified similarity cutoff for names)
#If no flowline names (gnis_name) match the source water name, then the closest flowline is returned
hydrolink.hydrolink_flowline

{'nhdhr flowline gnis name': 'Red Cedar River',
 'nhdhr flowline length km': 7.032,
 'nhdhr flowline permanent identifier': '152093413',
 'nhdhr flowline reachcode': '04050004000126',
 'flowline name similarity message': 'most likely match, based on fuzzy match',
 'cleaned source water name': 'the red cedar river',
 'flowline name similarity': 0.8823529411764706,
 'meters from flowline': 52.94060499992746,
 'nhdhr flowline measure': 29.511920388736257,
 'closest flowline order': 1}

In [7]:
import pandas as pd
df = pd.read_csv('nhdhr_hydrolink_output.csv')
df.head()

,source id,source lat nad83,source lon nad83,source buffer meters,closest conluence meters,closest flowline order,total count flowlines in buffer,source water name,cleaned source water name,flowline name similarity,...,nhdhr flowline gnis name,nhdhr flowline length km,nhdhr flowline permanent identifier,nhdhr flowline reachcode,meters from flowline,nhdhr flowline measure,nhdhr waterbody permanent identifier,nhdhr waterbody gnis name,nhdhr waterbody reachcode,hydrolink message
0,1,42.7284,-84.5026,2000,595.535732,1,9,The Red Cedar River,the red cedar river,0.882353,...,Red Cedar River,7.032,152093413,4050004000126,52.940605,29.51192,NaN,NaN,NaN,NaN
